In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
%ls '/kaggle/input/gtsrb-german-traffic-sign/Train/'

0/   11/  14/  17/  2/   22/  25/  28/  30/  33/  36/  39/  41/  6/  9/
1/   12/  15/  18/  20/  23/  26/  29/  31/  34/  37/  4/   42/  7/
10/  13/  16/  19/  21/  24/  27/  3/   32/  35/  38/  40/  5/   8/


In [2]:
print(pd.read_csv('/kaggle/input/gtsrb-german-traffic-sign/Train.csv').head())

   Width  Height  Roi.X1  Roi.Y1  Roi.X2  Roi.Y2  ClassId  \
0     27      26       5       5      22      20       20   
1     28      27       5       6      23      22       20   
2     29      26       6       5      24      21       20   
3     28      27       5       6      23      22       20   
4     28      26       5       5      23      21       20   

                             Path  
0  Train/20/00020_00000_00000.png  
1  Train/20/00020_00000_00001.png  
2  Train/20/00020_00000_00002.png  
3  Train/20/00020_00000_00003.png  
4  Train/20/00020_00000_00004.png  


In [3]:
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from torchvision import transforms
import matplotlib.pyplot as plt
import torch.optim as optim

train_trans = transforms.Compose([
    transforms.RandomRotation(9.6),
    transforms.RandomAutocontrast(),
    transforms.ToTensor(),
    transforms.Resize((128,128))
])

data_train = ImageFolder(
    '/kaggle/input/gtsrb-german-traffic-sign/Train',
    transform= train_trans
)

dataloader_train = DataLoader(
    data_train,
    shuffle = True,
    batch_size = 1000
)

#change dataloader batch size to 1 before running 
# image, label = next(iter(dataloader_train))
# print(image.shape)
# image = image.squeeze().permute(1,2,0)
# print(image.shape)
# plt.imshow(image)
# plt.show()

In [4]:
import torch.nn as nn
class TrafficSign(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.feature_extractor = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.ELU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(32, 64, kernel_size=3, padding =1),
            nn.ELU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Flatten()
        )
        self.classifier = nn.Linear(64*32*32, num_classes)
        
    def forward(self, x):
        x = self.feature_extractor(x)
        x= self.classifier(x)
        return x 
    
    

In [5]:
ts = TrafficSign(num_classes = 43).to('cuda')
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(ts.parameters(), lr=1e-3)

for epoch in range(10):
    running_loss = 0.0
    
    for images, labels in dataloader_train:
        optimizer.zero_grad()
        outputs = ts(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    
    epoch_loss = running_loss/len(dataloader_train)
    print(f'Epoch {epoch + 1} loss is {epoch_loss:.4f}')

AssertionError: Torch not compiled with CUDA enabled